# N-Queens

The n-queens puzzle is the problem of placing **n** queens on an **n x n** chessboard such that no two queens attack each other.

Given an integer **n**, return all distinct solutions to the **n-queens** puzzle. You may return the answer in any order.

Each solution contains a distinct board configuration of the **n-queens'** placement, where **'Q'** and **'.'** both indicate a queen and an empty space, respectively.



# Base Code

In [ ]:
def solveNQueens(n: int):
    # Track which columns and diagonals are already occupied by queens.
    # 'cols' holds used column indices c
    # 'diag' holds used main diagonals identified by (r - c)
    # 'anti' holds used anti-diagonals identified by (r + c)
    cols, diag, anti = set(), set(), set()

    # Mutable board for building solutions; "." = empty, "Q" = queen.
    # Using a list of lists makes it easy to place/remove queens.
    board = [["."] * n for _ in range(n)]

    # Collect all valid board configurations here (as list[str]).
    res = []

    def backtrack(r: int):

        #return the board visualizer (final step)
        # If we've placed queens in all rows, we found a full solution.
        #activate when the final answer of this problem is ready
        if r == n:
            # Join each row list into a string snapshot and save it.
            res.append(["".join(row) for row in board])
            return

        # Try to place a queen in each column of the current row r.
        for c in range(n):
            # Skip this column if it conflicts with an existing queen:
            # same column, same (r-c) diagonal, or same (r+c) anti-diagonal.
            if c in cols or (r - c) in diag or (r + c) in anti:
                continue

            # Choose: place a queen at (r, c) and mark the constraints.
            # A constraint is set by this c and r
            cols.add(c)
            diag.add(r - c)
            anti.add(r + c)
            board[r][c] = "Q"

            # Recurse to the next row.
            #we go one step deeper r -> r+1
            #if all goes when we go deeper and deeper successfuly and no backtrack happens
            #so we stay on this line (backtrack(r+1)) for all the deeper levels and last layer returns the condition of r==n
            backtrack(r + 1)

            # Undo (backtrack): remove the queen and unmark constraints.
            #we remove the contraint because we are backtracking
            board[r][c] = "."
            cols.remove(c)
            diag.remove(r - c)
            anti.remove(r + c)

    # Start placing queens from the first row.
    backtrack(0)
    return res

# example:
solutions = solveNQueens(4)
for b in solutions:
    print("\n".join(b), "\n")


.Q..
...Q
Q...
..Q. 

..Q.
Q...
...Q
.Q.. 



# Versbose Printing 

In [4]:
from typing import List, Set

class LimitReached(Exception):
    pass

class FirstSolutionFound(Exception):
    pass

def solve_n_queens_verbose(
    n: int,
    *,
    limit: int | None = None,          # max number of steps to print (None = unlimited)
    stop_after_first: bool = False      # stop as soon as a solution is found
) -> List[List[str]]:
    """
    Verbose N-Queens solver with step-by-step tracing.

    Actions:
      - TRY:       considering placing a queen at (r, c)
      - CONFLICT:  that (r, c) is unsafe (reason shown)
      - PLACE:     placed a queen at (r, c)
      - BACKTRACK: removed queen from (r, c) while unwinding
      - SOLUTION:  a full board (all rows filled) was found

    Vars:
      - cols: set of used columns
      - d1:   set of used major-diagonals (r - c)
      - d2:   set of used minor-diagonals (r + c)
      - board: list where board[r] = c (or -1 if empty)
    """
    solutions: List[List[str]] = []
    cols: Set[int] = set()
    d1: Set[int] = set()  # r - c
    d2: Set[int] = set()  # r + c
    board = [-1] * n
    step = 0

    def board_rows() -> List[str]:
        rows = []
        for rr in range(n):
            cc = board[rr]
            if cc == -1:
                rows.append("." * n)
            else:
                row = ["." for _ in range(n)]
                row[cc] = "Q"
                rows.append("".join(row))
        return rows

    def print_state(depth: int, action: str, r: int | None, c: int | None, note: str | None = None):
        nonlocal step
        step += 1
        indent = "  " * depth
        where = f" @ (r={r}, c={c})" if r is not None and c is not None else ""
        print(f"{indent}[{step}] {action}{where}")
        if note:
            print(f"{indent}    note: {note}")
        print(f"{indent}    cols={sorted(cols)}  d1(r-c)={sorted(d1)}  d2(r+c)={sorted(d2)}")
        print(f"{indent}    board={board}")
        for row in board_rows():
            print(f"{indent}      {row}")
        if limit is not None and step >= limit:
            raise LimitReached(f"Step limit reached ({limit}).")

    def snapshot_solution() -> List[str]:
        return board_rows()

    def backtrack(r: int, depth: int):
        # If all rows filled, we have a solution
        if r == n:
            print_state(depth, "SOLUTION", None, None, note="All rows placed")
            solutions.append(snapshot_solution())
            if stop_after_first:
                raise FirstSolutionFound
            return

        # Try all columns in this row
        for c in range(n):
            print_state(depth, "TRY", r, c)

            conflict_reasons = []
            if c in cols:
                conflict_reasons.append("same col")
            if (r - c) in d1:
                conflict_reasons.append("same diag1 (r-c)")
            if (r + c) in d2:
                conflict_reasons.append("same diag2 (r+c)")

            if conflict_reasons:
                print_state(depth, "CONFLICT", r, c, note=" & ".join(conflict_reasons))
                continue

            # Choose
            cols.add(c); d1.add(r - c); d2.add(r + c); board[r] = c
            print_state(depth, "PLACE", r, c)

            # Explore
            backtrack(r + 1, depth + 1)

            # Un-choose
            cols.remove(c); d1.remove(r - c); d2.remove(r + c); board[r] = -1
            print_state(depth, "BACKTRACK", r, c)

    print(f"--- N-Queens Backtracking Trace (n={n}) ---")
    try:
        backtrack(0, 0)
    except LimitReached as e:
        print(str(e))
    except FirstSolutionFound:
        print("--- Stopped after the first solution as requested ---")

    print(f"--- Found {len(solutions)} solution(s) ---")
    for i, sol in enumerate(solutions, 1):
        print(f"Solution {i}:")
        for row in sol:
            print("  " + row)
    return solutions


solve_n_queens_verbose(4,)

--- N-Queens Backtracking Trace (n=4) ---
[1] TRY @ (r=0, c=0)
    cols=[]  d1(r-c)=[]  d2(r+c)=[]
    board=[-1, -1, -1, -1]
      ....
      ....
      ....
      ....
[2] PLACE @ (r=0, c=0)
    cols=[0]  d1(r-c)=[0]  d2(r+c)=[0]
    board=[0, -1, -1, -1]
      Q...
      ....
      ....
      ....
  [3] TRY @ (r=1, c=0)
      cols=[0]  d1(r-c)=[0]  d2(r+c)=[0]
      board=[0, -1, -1, -1]
        Q...
        ....
        ....
        ....
  [4] CONFLICT @ (r=1, c=0)
      note: same col
      cols=[0]  d1(r-c)=[0]  d2(r+c)=[0]
      board=[0, -1, -1, -1]
        Q...
        ....
        ....
        ....
  [5] TRY @ (r=1, c=1)
      cols=[0]  d1(r-c)=[0]  d2(r+c)=[0]
      board=[0, -1, -1, -1]
        Q...
        ....
        ....
        ....
  [6] CONFLICT @ (r=1, c=1)
      note: same diag1 (r-c)
      cols=[0]  d1(r-c)=[0]  d2(r+c)=[0]
      board=[0, -1, -1, -1]
        Q...
        ....
        ....
        ....
  [7] TRY @ (r=1, c=2)
      cols=[0]  d1(r-c)=[0]  d2(r+c)=[0

[['.Q..', '...Q', 'Q...', '..Q.'], ['..Q.', 'Q...', '...Q', '.Q..']]